In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
#   Convert RGB images to CIE-LAB Color File

from __future__ import print_function
import argparse
import os

import numpy as np
import pickle
import random
from keras.preprocessing.image import list_pictures, array_to_img
from image_ext import list_pictures_in_multidir, load_imgs_asarray, img_dice_coeff
from fname_func import make_fnames, load_fnames

np.random.seed(2016)

/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def RGB2RGChroma(img):
    # convert RGB to RG Chromaticity Image
    # https://en.wikipedia.org/wiki/Rg_chromaticity
    ret = np.zeros((img.shape[0],img.shape[1],img.shape[2]),'float')
    
    R = img[:,:,0]
    G = img[:,:,1]
    B = img[:,:,2]
    I = R + G + B + 1
    r = (R+1)/I
    g = (G+1)/I
    b = (B+1)/I
    R2 = r*G / g
    B2 = ((1 - r - g)*G) / g
    
    ret[:,:,0] = r
    ret[:,:,1] = g
    ret[:,:,2] = 0
    
    return ret

In [3]:
def LabConversion(img):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2LAB)
    return ret

In [4]:
def LabAugumentation(img,L,a,b):
    ret = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2LAB)
    # print(ret)
    ret[:,:,0] = ret[:,:,0] + L
    ret[:,:,1] = ret[:,:,1] + a
    ret[:,:,2] = ret[:,:,2] + b
    ret = np.clip(ret,0,255)
    ret = cv2.cvtColor(ret, cv2.COLOR_LAB2BGR)    
    return ret

In [5]:
def ReflectionAugumentation(im, im_irismask, im_reflection, alpha):
    tmp = cv2.resize(im_reflection, (im.shape[1],im.shape[0]))
    mask = cv2.resize(im_irismask, (im.shape[1],im.shape[0]))
    mask[mask > 0] = 1
    im2 = im.astype(np.float64) + tmp.astype(np.float64)*mask.astype(np.float64)*alpha
    im2[im2 > 255] = 255
    return im2.astype(np.uint8)

In [6]:
#
#  MAIN STARTS FROM HERE
#
if __name__ == '__main__':
    
    target_size = (224, 224)
    dim_ordering = 'channels_last'
    
#    fnames = load_fnames('data.nnlab/list_files_all.txt')
#    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data.nnlab/image','data.nnlab/gt','')
#    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data.nnlab.LAB/img','data.nnlab/gt','')
    fnames = load_fnames('data/list_files_all.txt')
    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    [fpaths_xs_out,fpaths_ys_out] = make_fnames(fnames,'data_augumented_2/img','data_augumented_2/mask','OperatorA_')

    if os.path.exists('data_augumented_2') == 0:
        os.mkdir('data_augumented_2')
        os.mkdir('data_augumented_2/img')
        os.mkdir('data_augumented_2/mask')
    
    simulate_reflection = 1
    MAX_ALPHA = 0.25
    
    if simulate_reflection == 1:
        fnames_sc = load_fnames('data.scenes/list_files.txt')
        [fpaths_sc,tmp] = make_fnames(fnames_sc,'data.scenes/','data.scenes/','')
        [tmp,fpaths_irismask] = make_fnames(fnames,'data/img','data/mask_iris','OperatorA_') 
        num_reflection = len(fnames_sc)

In [7]:
    import matplotlib.pyplot as plt
    from PIL import Image 
    import cv2
    
    aug_div = [20,30,30]    
    
    fpath_img = 'data_augumented_2/img'
    fpath_mask = 'data_augumented_2/mask'

In [10]:
    for i in range(len(fpaths_xs_train)):
        img = cv2.imread(fpaths_xs_train[i])
        mask = cv2.imread(fpaths_ys_train[i])
        
        fnames_img = fpath_img + '/' + fnames[i];
        cv2.imwrite(fnames_img, img)
        
        fnames_mask = fpath_mask + '/' + fnames[i];
        print(fnames_mask)
        cv2.imwrite(fnames_mask,mask)
        
        im_irismask = cv2.imread(fpaths_irismask[i])
        
        for j in range(6):
            aug_param = [ random.randint(-aug_div[0],aug_div[0]), random.randint(-aug_div[1],aug_div[1]),
                             random.randint(-aug_div[2],aug_div[2]) ]
            print(aug_param)
            im = LabAugumentation(img,aug_param[0],aug_param[1],aug_param[2])
            
            if simulate_reflection == 1:
                im_reflection = cv2.imread(fpaths_sc[random.randint(1,num_reflection)-1])

                alpha = random.uniform(0,0.25)
                im2 = ReflectionAugumentation(im, im_irismask, im_reflection, alpha)
            
            fnames_img = '%s/%03d_%s'%(fpath_img, j, fnames[i])
            fnames_mask = '%s/%03d_%s'%(fpath_mask, j, fnames[i])
            print(fnames_img)
            print(fnames_mask)
            cv2.imwrite(fnames_img, im2)
            cv2.imwrite(fnames_mask, mask)
        

data_augumented_2/mask/C100_S1_I1.tiff
[-1, 4, 6]
data_augumented_2/img/000_C100_S1_I1.tiff
data_augumented_2/mask/000_C100_S1_I1.tiff
[-9, 23, 17]
data_augumented_2/img/001_C100_S1_I1.tiff
data_augumented_2/mask/001_C100_S1_I1.tiff
[-14, -14, 27]
data_augumented_2/img/002_C100_S1_I1.tiff
data_augumented_2/mask/002_C100_S1_I1.tiff
[-12, 17, -2]
data_augumented_2/img/003_C100_S1_I1.tiff
data_augumented_2/mask/003_C100_S1_I1.tiff
[14, -1, 17]
data_augumented_2/img/004_C100_S1_I1.tiff
data_augumented_2/mask/004_C100_S1_I1.tiff
[12, -5, -14]
data_augumented_2/img/005_C100_S1_I1.tiff
data_augumented_2/mask/005_C100_S1_I1.tiff
data_augumented_2/mask/C100_S1_I10.tiff
[-19, 13, -29]
data_augumented_2/img/000_C100_S1_I10.tiff
data_augumented_2/mask/000_C100_S1_I10.tiff
[-19, 19, -23]
data_augumented_2/img/001_C100_S1_I10.tiff
data_augumented_2/mask/001_C100_S1_I10.tiff
[2, 11, 22]
data_augumented_2/img/002_C100_S1_I10.tiff
data_augumented_2/mask/002_C100_S1_I10.tiff
[17, 19, 22]
data_augumented

data_augumented_2/img/001_C100_S1_I8.tiff
data_augumented_2/mask/001_C100_S1_I8.tiff
[0, 6, -26]
data_augumented_2/img/002_C100_S1_I8.tiff
data_augumented_2/mask/002_C100_S1_I8.tiff
[-20, 3, 8]
data_augumented_2/img/003_C100_S1_I8.tiff
data_augumented_2/mask/003_C100_S1_I8.tiff
[-11, 24, 23]
data_augumented_2/img/004_C100_S1_I8.tiff
data_augumented_2/mask/004_C100_S1_I8.tiff
[3, -21, 26]
data_augumented_2/img/005_C100_S1_I8.tiff
data_augumented_2/mask/005_C100_S1_I8.tiff
data_augumented_2/mask/C100_S1_I9.tiff
[-7, -27, -24]
data_augumented_2/img/000_C100_S1_I9.tiff
data_augumented_2/mask/000_C100_S1_I9.tiff
[0, -3, 8]
data_augumented_2/img/001_C100_S1_I9.tiff
data_augumented_2/mask/001_C100_S1_I9.tiff
[10, 5, -17]
data_augumented_2/img/002_C100_S1_I9.tiff
data_augumented_2/mask/002_C100_S1_I9.tiff
[2, -1, -7]
data_augumented_2/img/003_C100_S1_I9.tiff
data_augumented_2/mask/003_C100_S1_I9.tiff
[3, -2, -15]
data_augumented_2/img/004_C100_S1_I9.tiff
data_augumented_2/mask/004_C100_S1_I9.t

data_augumented_2/img/001_C100_S2_I6.tiff
data_augumented_2/mask/001_C100_S2_I6.tiff
[10, 1, 14]
data_augumented_2/img/002_C100_S2_I6.tiff
data_augumented_2/mask/002_C100_S2_I6.tiff
[10, 2, -22]
data_augumented_2/img/003_C100_S2_I6.tiff
data_augumented_2/mask/003_C100_S2_I6.tiff
[-8, 29, -27]
data_augumented_2/img/004_C100_S2_I6.tiff
data_augumented_2/mask/004_C100_S2_I6.tiff
[3, 30, -8]
data_augumented_2/img/005_C100_S2_I6.tiff
data_augumented_2/mask/005_C100_S2_I6.tiff
data_augumented_2/mask/C100_S2_I7.tiff
[14, 7, -23]
data_augumented_2/img/000_C100_S2_I7.tiff
data_augumented_2/mask/000_C100_S2_I7.tiff
[10, 9, -25]
data_augumented_2/img/001_C100_S2_I7.tiff
data_augumented_2/mask/001_C100_S2_I7.tiff
[20, 16, 26]
data_augumented_2/img/002_C100_S2_I7.tiff
data_augumented_2/mask/002_C100_S2_I7.tiff
[3, -20, -19]
data_augumented_2/img/003_C100_S2_I7.tiff
data_augumented_2/mask/003_C100_S2_I7.tiff
[-3, -17, -29]
data_augumented_2/img/004_C100_S2_I7.tiff
data_augumented_2/mask/004_C100_S2_

data_augumented_2/img/001_C101_S1_I4.tiff
data_augumented_2/mask/001_C101_S1_I4.tiff
[-19, -20, -3]
data_augumented_2/img/002_C101_S1_I4.tiff
data_augumented_2/mask/002_C101_S1_I4.tiff
[0, 1, -28]
data_augumented_2/img/003_C101_S1_I4.tiff
data_augumented_2/mask/003_C101_S1_I4.tiff
[-9, -20, 8]
data_augumented_2/img/004_C101_S1_I4.tiff
data_augumented_2/mask/004_C101_S1_I4.tiff
[-2, -24, 13]
data_augumented_2/img/005_C101_S1_I4.tiff
data_augumented_2/mask/005_C101_S1_I4.tiff
data_augumented_2/mask/C101_S1_I5.tiff
[10, 5, 30]
data_augumented_2/img/000_C101_S1_I5.tiff
data_augumented_2/mask/000_C101_S1_I5.tiff
[-1, -11, -29]
data_augumented_2/img/001_C101_S1_I5.tiff
data_augumented_2/mask/001_C101_S1_I5.tiff
[-20, -15, 10]
data_augumented_2/img/002_C101_S1_I5.tiff
data_augumented_2/mask/002_C101_S1_I5.tiff
[-5, 26, -5]
data_augumented_2/img/003_C101_S1_I5.tiff
data_augumented_2/mask/003_C101_S1_I5.tiff
[13, -12, -6]
data_augumented_2/img/004_C101_S1_I5.tiff
data_augumented_2/mask/004_C101

data_augumented_2/img/001_C101_S2_I2.tiff
data_augumented_2/mask/001_C101_S2_I2.tiff
[12, -14, -12]
data_augumented_2/img/002_C101_S2_I2.tiff
data_augumented_2/mask/002_C101_S2_I2.tiff
[14, 1, -27]
data_augumented_2/img/003_C101_S2_I2.tiff
data_augumented_2/mask/003_C101_S2_I2.tiff
[10, 8, -29]
data_augumented_2/img/004_C101_S2_I2.tiff
data_augumented_2/mask/004_C101_S2_I2.tiff
[-10, -25, -15]
data_augumented_2/img/005_C101_S2_I2.tiff
data_augumented_2/mask/005_C101_S2_I2.tiff
data_augumented_2/mask/C101_S2_I3.tiff
[-1, -23, -14]
data_augumented_2/img/000_C101_S2_I3.tiff
data_augumented_2/mask/000_C101_S2_I3.tiff
[-20, -7, 24]
data_augumented_2/img/001_C101_S2_I3.tiff
data_augumented_2/mask/001_C101_S2_I3.tiff
[-12, 7, 11]
data_augumented_2/img/002_C101_S2_I3.tiff
data_augumented_2/mask/002_C101_S2_I3.tiff
[1, -18, 14]
data_augumented_2/img/003_C101_S2_I3.tiff
data_augumented_2/mask/003_C101_S2_I3.tiff
[-9, 9, 7]
data_augumented_2/img/004_C101_S2_I3.tiff
data_augumented_2/mask/004_C101

...

data_augumented_2/img/000_C99_S1_I14.tiff
data_augumented_2/mask/000_C99_S1_I14.tiff
[-9, -11, -13]
data_augumented_2/img/001_C99_S1_I14.tiff
data_augumented_2/mask/001_C99_S1_I14.tiff
[-19, 22, -10]
data_augumented_2/img/002_C99_S1_I14.tiff
data_augumented_2/mask/002_C99_S1_I14.tiff
[0, -3, -22]
data_augumented_2/img/003_C99_S1_I14.tiff
data_augumented_2/mask/003_C99_S1_I14.tiff
[2, -5, -1]
data_augumented_2/img/004_C99_S1_I14.tiff
data_augumented_2/mask/004_C99_S1_I14.tiff
[-11, -4, 20]
data_augumented_2/img/005_C99_S1_I14.tiff
data_augumented_2/mask/005_C99_S1_I14.tiff
data_augumented_2/mask/C99_S1_I15.tiff
[-18, 30, 22]
data_augumented_2/img/000_C99_S1_I15.tiff
data_augumented_2/mask/000_C99_S1_I15.tiff
[-12, 27, 1]
data_augumented_2/img/001_C99_S1_I15.tiff
data_augumented_2/mask/001_C99_S1_I15.tiff
[-4, -27, -12]
data_augumented_2/img/002_C99_S1_I15.tiff
data_augumented_2/mask/002_C99_S1_I15.tiff
[-20, 21, -6]
data_augumented_2/img/003_C99_S1_I15.tiff
data_augumented_2/mask/003_C9

data_augumented_2/img/002_C99_S2_I12.tiff
data_augumented_2/mask/002_C99_S2_I12.tiff
[17, -21, 12]
data_augumented_2/img/003_C99_S2_I12.tiff
data_augumented_2/mask/003_C99_S2_I12.tiff
[-18, 17, -29]
data_augumented_2/img/004_C99_S2_I12.tiff
data_augumented_2/mask/004_C99_S2_I12.tiff
[19, 17, -29]
data_augumented_2/img/005_C99_S2_I12.tiff
data_augumented_2/mask/005_C99_S2_I12.tiff
data_augumented_2/mask/C99_S2_I13.tiff
[-15, 0, -12]
data_augumented_2/img/000_C99_S2_I13.tiff
data_augumented_2/mask/000_C99_S2_I13.tiff
[9, 9, 17]
data_augumented_2/img/001_C99_S2_I13.tiff
data_augumented_2/mask/001_C99_S2_I13.tiff
[18, 24, 28]
data_augumented_2/img/002_C99_S2_I13.tiff
data_augumented_2/mask/002_C99_S2_I13.tiff
[-10, -18, -11]
data_augumented_2/img/003_C99_S2_I13.tiff
data_augumented_2/mask/003_C99_S2_I13.tiff
[6, 8, 23]
data_augumented_2/img/004_C99_S2_I13.tiff
data_augumented_2/mask/004_C99_S2_I13.tiff
[13, 29, 25]
data_augumented_2/img/005_C99_S2_I13.tiff
data_augumented_2/mask/005_C99_S2

data_augumented_2/img/004_C9_S1_I10.tiff
data_augumented_2/mask/004_C9_S1_I10.tiff
[4, -25, -5]
data_augumented_2/img/005_C9_S1_I10.tiff
data_augumented_2/mask/005_C9_S1_I10.tiff
data_augumented_2/mask/C9_S1_I11.tiff
[-12, 30, -26]
data_augumented_2/img/000_C9_S1_I11.tiff
data_augumented_2/mask/000_C9_S1_I11.tiff
[5, -21, 12]
data_augumented_2/img/001_C9_S1_I11.tiff
data_augumented_2/mask/001_C9_S1_I11.tiff
[-6, 16, -14]
data_augumented_2/img/002_C9_S1_I11.tiff
data_augumented_2/mask/002_C9_S1_I11.tiff
[-16, -16, -19]
data_augumented_2/img/003_C9_S1_I11.tiff
data_augumented_2/mask/003_C9_S1_I11.tiff
[-13, -2, 0]
data_augumented_2/img/004_C9_S1_I11.tiff
data_augumented_2/mask/004_C9_S1_I11.tiff
[3, 16, 9]
data_augumented_2/img/005_C9_S1_I11.tiff
data_augumented_2/mask/005_C9_S1_I11.tiff
data_augumented_2/mask/C9_S1_I12.tiff
[-18, 29, -3]
data_augumented_2/img/000_C9_S1_I12.tiff
data_augumented_2/mask/000_C9_S1_I12.tiff
[-18, -7, 11]
data_augumented_2/img/001_C9_S1_I12.tiff
data_augument

data_augumented_2/img/001_C9_S2_I1.tiff
data_augumented_2/mask/001_C9_S2_I1.tiff
[8, -28, 25]
data_augumented_2/img/002_C9_S2_I1.tiff
data_augumented_2/mask/002_C9_S2_I1.tiff
[13, -13, -17]
data_augumented_2/img/003_C9_S2_I1.tiff
data_augumented_2/mask/003_C9_S2_I1.tiff
[11, 16, -9]
data_augumented_2/img/004_C9_S2_I1.tiff
data_augumented_2/mask/004_C9_S2_I1.tiff
[5, 6, -24]
data_augumented_2/img/005_C9_S2_I1.tiff
data_augumented_2/mask/005_C9_S2_I1.tiff
data_augumented_2/mask/C9_S2_I10.tiff
[-2, 16, 19]
data_augumented_2/img/000_C9_S2_I10.tiff
data_augumented_2/mask/000_C9_S2_I10.tiff
[-7, -13, 15]
data_augumented_2/img/001_C9_S2_I10.tiff
data_augumented_2/mask/001_C9_S2_I10.tiff
[13, 16, -19]
data_augumented_2/img/002_C9_S2_I10.tiff
data_augumented_2/mask/002_C9_S2_I10.tiff
[15, 6, 14]
data_augumented_2/img/003_C9_S2_I10.tiff
data_augumented_2/mask/003_C9_S2_I10.tiff
[16, 7, 3]
data_augumented_2/img/004_C9_S2_I10.tiff
data_augumented_2/mask/004_C9_S2_I10.tiff
[-12, -20, -30]
data_augu

data_augumented_2/img/004_C9_S2_I8.tiff
data_augumented_2/mask/004_C9_S2_I8.tiff
[8, -14, -26]
data_augumented_2/img/005_C9_S2_I8.tiff
data_augumented_2/mask/005_C9_S2_I8.tiff
data_augumented_2/mask/C9_S2_I9.tiff
[-17, -14, -4]
data_augumented_2/img/000_C9_S2_I9.tiff
data_augumented_2/mask/000_C9_S2_I9.tiff
[10, 21, 28]
data_augumented_2/img/001_C9_S2_I9.tiff
data_augumented_2/mask/001_C9_S2_I9.tiff
[-12, 19, -1]
data_augumented_2/img/002_C9_S2_I9.tiff
data_augumented_2/mask/002_C9_S2_I9.tiff
[4, 1, -13]
data_augumented_2/img/003_C9_S2_I9.tiff
data_augumented_2/mask/003_C9_S2_I9.tiff
[-3, 23, 12]
data_augumented_2/img/004_C9_S2_I9.tiff
data_augumented_2/mask/004_C9_S2_I9.tiff
[-17, 8, -27]
data_augumented_2/img/005_C9_S2_I9.tiff
data_augumented_2/mask/005_C9_S2_I9.tiff
